This code will generate .bed, .bim, .fam files for the patients in the test set in the windows of each gene in the test expression file

In [1]:
# PATH TO DIRECTORY CONTAINING IMPUTED TCGA GENOTYPES
TCGA_GENO='/cellar/users/jlz014/Data/TCGA/TCGA_EUR_imputed'

In [ ]:
# PATH TO TEST GENE EXPRESSION MATRIX:
PRE_GEXP_DIR='/data/cellardata/users/jlz014/TCGA/expression/'
PRE_GEXP="TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.for_predictions.txt"

# PATH TO PREFIX FOR GEUVADIS GENOTYPES SPLIT BY CHROMOSOME
# SUBSAMPLE THESE TO THE LDREF SNPS FOR EFFICIENCY
PRE_GENO="EUR_imputed_females_filtered"

# PATH TO OUTPUT DIRECTORY 
OUT_DIR="./brca_test_patients_tmp_plink"
if [[ ! -e $OUT_DIR ]]; then
    mkdir -p $OUT_DIR
fi

# ROWS IN THE MATRIX TO ANALYZE (FOR BATCHED RUNS)
BATCH_START=1
BATCH_END=10

# make *.ID for gene expression file
echo `awk 'NR==1' $PRE_GEXP_DIR$PRE_GEXP` | tr ' ' '\n' | tail -n+5 > $PRE_GEXP.ID.tmp
paste $PRE_GEXP.ID.tmp $PRE_GEXP.ID.tmp | column -s $'\t' -t > $PRE_GEXP.ID
rm $PRE_GEXP.ID.tmp

In [4]:
head $PRE_GEXP.ID

TCGA-OL-A5RZ  TCGA-OL-A5RZ
TCGA-C8-A27A  TCGA-C8-A27A
TCGA-BH-A2L8  TCGA-BH-A2L8
TCGA-AR-A5QP  TCGA-AR-A5QP
TCGA-E9-A1RE  TCGA-E9-A1RE
TCGA-S3-AA0Z  TCGA-S3-AA0Z
TCGA-E9-A1R6  TCGA-E9-A1R6
TCGA-BH-A0DG  TCGA-BH-A0DG
TCGA-AQ-A1H3  TCGA-AQ-A1H3
TCGA-BH-A0H0  TCGA-BH-A0H0


In [ ]:
# --- BEGIN SCRIPT:
NR="${BATCH_START}_${BATCH_END}"
# mkdir $OUT_DIR
mkdir --parents $OUT_DIR/1_10

cat $PRE_GEXP_DIR$PRE_GEXP | awk -vs=$BATCH_START -ve=$BATCH_END 'NR > s && NR <= e' |  while read PARAM; do
# PARAM=`awk 'NR==14808' $PRE_GEXP_DIR$PRE_GEXP`

# Get the gene positions +/- 500kb (1MB window)
CHR=`echo $PARAM | awk '{ print $2 }'`
P0=`echo $PARAM | awk '{ print $3 - 0.5e6 }'`
P1=`echo $PARAM | awk '{ print $4 + 0.5e6 }'`
GNAME=`echo $PARAM | awk '{ print $1 }'`

# # output directory for tmp plink files generated 
OUT="$OUT_DIR/$NR/$PRE_GEXP.$GNAME"
echo $GNAME $CHR $P0 $P1
echo $OUT

# Pull out the current gene expression phenotype
echo $PARAM | tr ' ' '\n' | tail -n+5 | paste $PRE_GEXP.ID - > $OUT.pheno

echo 'running plink'
# # Get the locus genotypes for all samples and set current gene expression as the phenotype
plink2 --bfile $TCGA_GENO/$PRE_GENO.$CHR --pheno $OUT.pheno --make-bed --out $OUT --keep $OUT.pheno --chr $CHR --from-bp $P0 --to-bp $P1 --extract $TCGA_GENO/$PRE_GENO.$CHR.bim

echo 'done'
done